In [1]:
import pandas as pd
import re, os 
import datetime
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

os.listdir('./data_down')


"""
download data without empty ou, when select
['bca_cc_cust_rel.csv',
 'bca_without_mail.csv',
 'dwd_dsc_d365_contract_df.csv',
 'dwd_dsc_d365_opportunity_df.csv',
 'dwd_dsc_finance_estimate_summary_di.csv',
 'dwd_dsc_hr_cost_dtl.csv',
 'dwd_fact_warehouse_billing_detail_dtl.csv']
"""

"\ndownload data without empty ou, when select\n['bca_cc_cust_rel.csv',\n 'bca_without_mail.csv',\n 'dwd_dsc_d365_contract_df.csv',\n 'dwd_dsc_d365_opportunity_df.csv',\n 'dwd_dsc_finance_estimate_summary_di.csv',\n 'dwd_dsc_hr_cost_dtl.csv',\n 'dwd_fact_warehouse_billing_detail_dtl.csv']\n"

In [183]:
# read data
df_oppa = pd.read_csv('./data_down/' + 'dwd_dsc_d365_opportunity_df.csv', sep = '\001') 
df_esti = pd.read_csv('./data_down/' + 'dwd_dsc_finance_estimate_summary_di.csv', sep = '\001') 
df_cost = pd.read_csv('./data_down/' + 'dwd_dsc_hr_cost_dtl.csv', sep = '\001') 
df_bill = pd.read_csv('./data_down/' + 'dwd_fact_warehouse_billing_detail_dtl.csv', sep = '\001') 
df_bcaw = pd.read_csv('./data_down/' + 'bca_without_mail.csv', sep = '\001') 
rel = pd.read_csv('./data_down/' + 'bca_cc_cust_rel.csv', sep = '\001') 
# for i in os.listdir('./data_down'): 
#     print(re.findall( '\w+', i)[0])
#     df = pd.read_csv('./data_down/' + i, sep = '\001') 
def colname_modi(df):
    df.columns = [ re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
for i in [df_oppa, df_bill,df_cost,df_esti,rel, df_bcaw]:
    colname_modi(i)
 
rel = rel.drop(['cc_in_bca', 'start_date'], axis=1)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_19972/1335000026.py:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  frame = pd.DataFrame(df_cost.sum(axis = 0) == 0)


In [215]:
def data_prepare():
    """
    data prepare opportunity .
    """
    df = pd.read_csv('./data_down/dwd_dsc_d365_opportunity_df.csv', sep = '\001')

    df.columns = [re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
    df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
    colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']
    for i in colname:
        df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

    df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
    df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])

    contract_summary = df.groupby([
        'account_name_en', 'contract_end_date', 'account_id','bg', 'contract_term', 'milestone','opportunity_record'\
        ,'product1', 'product2', 'sector'
        ])[[
        'annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue', 'contract_value']].sum()
    contract_summary = contract_summary.reset_index()
    contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)
    return contract_summary

opportunity = data_prepare();opportunity.head(3)
# del contract_summary

,account_name_en,contract_end_date,account_id,bg,contract_term,milestone,opportunity_record,product1,product2,sector,annual_average_gross_profit_base,annual_average_gross_profit,annual_average_revenue_base,annual_average_revenue,contract_value,contract_end_yr
0,ABC,2021-06-28,e0209d33-e3aa-ea11-a812-000d3a851174,TSC,12.0,Closed-Canceled,New Business,Warehousing,Transportation,Technology,5000.0,5000.0,50000.0,50000.0,50000.0,2021
1,ADIDAS,2018-09-29,f64a8fa5-a304-4c85-9641-7205e77e70d8,HK,36.0,Contract Signed,New Business,Warehousing,Warehousing,Retail,950.4,1188.0,10777.6,13472.0,40416.0,2018
2,ADIDAS,2019-10-30,f64a8fa5-a304-4c85-9641-7205e77e70d8,HK,12.0,Contract Signed,Straightforward Renewals,Warehousing,Transportation,Retail,1087.2,1359.0,13123.2,16404.0,0.0,2019


In [17]:
df_list = [opportunity,df_bill,df_cost,df_esti]


In [185]:
def check_ouname(df):
    print(pd.Series(df.columns).str.extract('(ou_code|cost_center)').dropna(),'\t')

for i in df_list:
    check_ouname(i)

del df_list,i

Empty DataFrame
Columns: [0]
Index: [] 	
              0
13  cost_center 	
         0
1  ou_code 	
              0
12  cost_center 	


In [186]:
"""
to customer level. functions set up.
"""
def data_clean(df, substr_year, **kwargs):
    df = df[~df['account_name_en'].isna()]
    df[substr_year] = df[substr_year].astype(str).str.slice(0,4)
    df = df.groupby(['account_name_en','account_name_cn', substr_year]).agg({
        **kwargs
    })
    return df 

def to_cust_level(df,  substr_year, **kwargs):
    """
    merge all to cust level
    """
    try:
        df1 = df.merge(rel, left_on = 'ou_code', right_on = 'cc_in_ou', how  ='left')
    except:
        df1 = df.merge(rel, left_on = 'cost_center', right_on = 'cc_in_ou', how  ='left')
    return data_clean(df1, substr_year,  **kwargs)

In [216]:
df_bill['bms_bill_end_date'].str.slice()

,bms_bill_id,bms_order_id,bms_fee_type,bms_fee_name,bms_amount_include_tax,bms_amount,bms_tax_rate,bms_template_ids,bms_tax_included,bms_is_settlement_fix_valuation,bms_bill_start_date,bms_bill_end_date,cost_center
0,DSC1000000000016,NaN,other_short_barge_transport_fee,其他短驳运输费,4.0,4.000000,0.0000,"[""""]",1.0,2.0,20210613,20210812,QA-TEST-PRO-hetong
1,DSC1000000000016,NaN,other_short_barge_transport_fee,其他短驳运输费,2.0,2.000000,0.0000,[],1.0,1.0,20210613,20210812,QA-TEST-PRO-hetong
2,DSC1000000000017,NaN,claim_account_deduction_fee,索赔帐扣费,6.0,5.940594,0.0100,"[""""]",1.0,2.0,20210613,20210812,QA-TEST-PRO-hetong
3,DSC1000000000018,NaN,short_barge_transport_fee,短驳运输费,2.0,1.980198,0.0100,[],1.0,1.0,20210613,20210812,QA-TEST-PRO-hetong
4,DSC1000000000018,NaN,short_barge_transport_fee,短驳运输费,3.0,2.970297,0.0100,[],1.0,1.0,20210613,20210812,QA-TEST-PRO-hetong
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124693,M202111100000000014,NaN,other_value_added_fee,其他增值服务费,0.0,0.000000,0.0660,"[""T2392193601""]",0.0,2.0,20211024,20211026,STARBSHHTS
124694,M202111100000000014,NaN,other_value_added_fee,其他增值服务费,0.0,0.000000,0.0660,"[""T2392193601""]",0.0,2.0,20211024,20211026,STARBSHHTS
124695,M202111100000000014,NaN,other_value_added_fee,其他增值服务费,0.0,0.000000,0.0660,"[""T2392193601""]",0.0,2.0,20211024,20211026,STARBSHHTS
124696,M202111100000000014,NaN,other_value_added_fee,其他增值服务费,0.0,0.000000,0.0660,"[""T2392193601""]",0.0,2.0,20211024,20211026,STARBSHHTS


In [209]:
"""
~billing~
bill detail table now having 50+ bms_types. which is unacceptalble/
"""
df_bill.pivot_table(index= ['cost_center'],\
    columns= 'bms_fee_name',
    values='bms_amount',).reset_index()

bms_fee_name,cost_center,IT设备,人力费用开口收费,仓储费,代垫税费,代理费,入库费,其他场地费,其他增值服务费,其他报关费,...,空调设备,索赔及返利费用开口收费,索赔帐扣费,翻拍费,能源费,设备使用费用开口收费,货架,贴标费,贴标费用,返利费
0,ABBOTBJXWS,NaN,NaN,51604.539026,NaN,NaN,NaN,NaN,1313.402045,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABBOTCDXXS,NaN,NaN,38633.061123,NaN,NaN,NaN,NaN,1477.307000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABBOTSHDBP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16954.626245,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36317.263871,NaN,NaN
3,ABBOTSHDBS,NaN,NaN,414514.740566,NaN,NaN,NaN,NaN,34787.198113,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABBOTSHKQP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82705.485786,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7361.502717,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,dsc-test-01-PRO-FCY_testQA,NaN,NaN,0.980392,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352,dsc-test-01-cus,NaN,NaN,NaN,NaN,NaN,83.009901,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2383.17757,NaN
353,fcy-test-pro,NaN,NaN,5.715121,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.300297,NaN,NaN,NaN,NaN,NaN,NaN,5.555545
354,fcy-test-pro_testQA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.925926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
# to_cust_level(df = df_bill, substr_year = 'bms_bill_end_date' ,\
#     bms_amount = 'sum', bms_fee_type = [set,'nunique'], bms_fee_name = set, bms_bill_id = 'nunique')

In [213]:
# def tst(substr_year,**kwargs):
#     df_bill[substr_year] = df_bill[substr_year].astype(str).str.slice(0,4)
#     return  df_bill.groupby([substr_year, 'cost_center']).agg({
#         **kwargs
#     }).reset_index()

# tst(substr_year = 'bms_bill_end_date' ,\
#     bms_amount = 'sum', bms_fee_type = [set,'nunique'], bms_fee_name = set, bms_bill_id = 'nunique')

In [212]:
# test = df_bill.head(39)
# test.pivot_table(index= ['bms_bill_id', 'cost_center'],\
#     columns= ['bms_fee_type'],
#     values='bms_amount',)

In [211]:
"""
# cost
"""
df_cost = df_cost.dropna(axis=1, how = 'all').fillna(0)
frame = pd.DataFrame(df_cost.sum(axis = 0) == 0)
df_cost = df_cost.loc[:, frame[frame[0] == False].index.tolist()]
del frame
df_cost.head()

,id,ou_code,level,month,rev,gp,hr_cost,staff_cost,staff_cost_dir,hour_dir,...,staff_cost_bizos,staff_cost_bizos_actual,staff_roi_nmom,staff_cost_cm_mom,staff_cost_cm_nmom,rev_target,staff_cost_target,gp_target,hc_target,src_inc_day
0,1907,ABBOTBJXWS,site,201912,150.0,-66.0,53745.36,59.0,59.0,1144.0,...,357.72,357.72,4.85,3.77,-6.52,0.00,0.00,0.00,0.0,20211111
1,10047,ABBOTBJXWS,site,202102,119.0,-45.0,52033.54,54.0,0.0,0.0,...,514.18,514.18,-3.38,-6.43,-0.90,141.53,53.33,-44.43,0.0,20211111
2,6079,ABBOTBJXWS,site,202006,115.0,-53.0,45693.69,38.0,0.0,0.0,...,810.23,810.23,-12.92,0.32,3.13,0.00,0.00,0.00,0.0,20211111
3,1107,ABBOTBJXWS,site,201907,100.0,-140.0,45630.13,48.0,48.0,0.0,...,369.16,369.16,-4.56,0.25,-4.45,0.00,0.00,0.00,0.0,20211111
4,10560,ABBOTBJXWS,site,202103,65.0,-75.0,51766.79,54.0,0.0,0.0,...,514.18,514.18,-13.10,-0.52,-4.98,159.96,53.27,-25.95,0.0,20211111


In [ ]:
min = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
max = pd.to_datetime('2024-01-01', format='%Y-%m-%d')

In [ ]:
# fig = go.Figure(data = [go.Scatter(x = df_sub ['contract_end_date'] ,
#                                 y = df_sub ['annual_average_revenue'],
#                                 mode='markers',
#                                 marker_size=df_sub['annual_average_revenue']/1000,
#                                 opacity=0.87,
#                                 name='account',
#                                 textposition='bottom center')],
#                                 layout=go.Layout(
#                                     xaxis = dict(range = [min, max]),
#                                     yaxis = dict(range = [0, 100000]),
#                                     legend = {'xanchor': 'left'}
#                                 )
#                                 ) 
# fig

In [ ]:
# df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
# colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']
# for i in colname:
#     df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

# df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
# df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])
# contract_summary = df.groupby(['account_name_en', 'contract_end_date'])[[
#        'annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']].sum()
# contract_summary = contract_summary.reset_index()
# contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)

In [ ]:
contract_summary['contract_end_date']

In [ ]:
# fig = px.scatter(x = "contract_end_date", y= "annual_average_revenue", \
#     color = 'account_name_en',  size = 'annual_average_revenue',
#     data_frame  = contract_summary.head(65))

# min = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
# max = pd.to_datetime('2024-01-01', format='%Y-%m-%d')
# fig.update_xaxes(range = list([min, max]))

# fig.update_layout(
#     title_text="Title",
#     showlegend = True
# )

# # Add range slider
# fig.update_layout(

# )

